In [1]:
from google.colab import drive

drive.mount('/content/drive')

#!pip install --upgrade openai


Mounted at /content/drive


In [2]:
!pip install -q transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.6 MB/s eta 0:00:00


In [3]:
from openai import OpenAI
from google.colab import userdata

# Initialize DeepSeek client
deepseek_client = OpenAI(
    base_url='https://api.deepseek.com',
    api_key=userdata.get("DEEPSEEK_API_KEY")
)

# Sample tweet to test
sample_tweet = "I can’t believe how fast this year is flying by 😩"

# Construct prompt
system_message = "You are a helpful assistant that rephrases tweets using different words while keeping the tone and meaning intact."
prompt = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": f'Paraphrase this tweet: "{sample_tweet}"'}
]

# Make API call
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompt,
    max_tokens=60
)

# Extract the result
paraphrased = response.choices[0].message.content.strip()

# Show it
print("🟡 Original Tweet:")
print(sample_tweet)
print("\n🟢 Paraphrased by DeepSeek:")
print(paraphrased)


🟡 Original Tweet:
I can’t believe how fast this year is flying by 😩

🟢 Paraphrased by DeepSeek:
"Wow, this year is zooming by way too quickly 😫"  

Let me know if you'd like any adjustments!


In [4]:
import pandas as pd

# Load your original tweet dataset
column_names = ['target','ids','date','flag','user','text']
df = pd.read_csv("/content/drive/MyDrive/fp/training.1600000.processed.noemoticon.csv", encoding='latin-1', names=column_names)

# Reproduce the original shuffling (random_state=42)
df_sampled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract your GPT chunk (middle third)
chunk_3 = df_sampled.iloc[60000:90000].copy()


In [5]:
from openai import OpenAI
from google.colab import userdata

# Initialize DeepSeek client
deepseek_client = OpenAI(
    base_url='https://api.deepseek.com',
    api_key=userdata.get("DEEPSEEK_API_KEY")
)

# Sample tweet to test
sample_tweet = "I can’t believe how fast this year is flying by 😩"

# Construct prompt
system_message = "You are a helpful assistant that rephrases tweets using different words while keeping the tone and meaning intact."
prompt = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": f'Paraphrase this tweet: "{sample_tweet}"'}
]

# Make API call with max_tokens = 40
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompt,
    max_tokens=40  # ✅ updated here
)

# Extract the result
paraphrased = response.choices[0].message.content.strip()

# Show it
print("🟡 Original Tweet:")
print(sample_tweet)
print("\n🟢 Paraphrased by DeepSeek:")
print(paraphrased)


🟡 Original Tweet:
I can’t believe how fast this year is flying by 😩

🟢 Paraphrased by DeepSeek:
"Time is zooming by way too quickly this year 😫"  

Let me know if you'd like any adjustments!


In [ ]:
from openai import OpenAI
from google.colab import userdata
import os, json
from tqdm import tqdm

# ✅ Initialize DeepSeek client
deepseek_client = OpenAI(
    base_url='https://api.deepseek.com',
    api_key=userdata.get("DEEPSEEK_API_KEY")
)

# ✅ Configs
system_message = "You are a helpful assistant that rephrases tweets using different words while keeping the tone and meaning intact."
prompt_prefix = "Paraphrase this tweet: "
max_tokens = 40
batch_size = 100

# ✅ File paths
save_dir = "/content/drive/MyDrive/fp/deepseek_outputs/"
os.makedirs(save_dir, exist_ok=True)
output_file = os.path.join(save_dir, "chunk3_deepseek_paraphrased.jsonl")
index_file = os.path.join(save_dir, "chunk3_processed_indices.txt")

# ✅ Load progress
if os.path.exists(index_file):
    with open(index_file, "r") as f:
        processed = set(map(int, f.read().splitlines()))
else:
    processed = set()

# ✅ Batch variables
batch = []
batch_indices = []

# ✅ Process in batches
for idx, row in enumerate(tqdm(chunk_3.itertuples(), total=len(chunk_3))):
    if idx in processed:
        continue

    tweet = row.text.strip()

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f'{prompt_prefix}"{tweet}"'}
    ]

    try:
        response = deepseek_client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            max_tokens=max_tokens
        )
        paraphrased = response.choices[0].message.content.strip()

        batch.append({
            "id": idx,
            "original": tweet,
            "paraphrased": paraphrased
        })
        batch_indices.append(idx)

    except Exception as e:
        print(f"❌ Error on idx {idx}: {e}")
        continue

    # ✅ Save every 100 tweets
    if len(batch) == batch_size:
        with open(output_file, "a") as fout:
            for item in batch:
                fout.write(json.dumps(item) + "\n")

        with open(index_file, "a") as fidx:
            for index in batch_indices:
                fidx.write(f"{index}\n")

        print(f"💾 Saved batch of {batch_size} tweets at index {idx + 1}")

        # Reset batch
        batch = []
        batch_indices = []

# ✅ Save remaining tweets (if any)
if batch:
    with open(output_file, "a") as fout:
        for item in batch:
            fout.write(json.dumps(item) + "\n")

    with open(index_file, "a") as fidx:
        for index in batch_indices:
            fidx.write(f"{index}\n")

    print(f"💾 Saved final batch of {len(batch)} tweets")


 72%|███████▏  | 21617/30000 [09:40<14:34:45,  6.26s/it]

💾 Saved batch of 100 tweets at index 21617


 72%|███████▏  | 21717/30000 [19:14<12:49:11,  5.57s/it]

💾 Saved batch of 100 tweets at index 21717


 73%|███████▎  | 21817/30000 [28:49<13:20:53,  5.87s/it]

...

 74%|███████▍  | 22317/30000 [1:16:03<11:38:25,  5.45s/it]

💾 Saved batch of 100 tweets at index 22317


 75%|███████▍  | 22417/30000 [1:25:26<11:32:33,  5.48s/it]

💾 Saved batch of 100 tweets at index 22417


 75%|███████▍  | 22466/30000 [1:29:58<11:20:24,  5.42s/it]